# Programme de Matching

Regarder si l'établissement est fermé ou pas

## Preparation fichier

### Variable Ville

- process:
    - creer variables avec numeric seulement
    - recreer ville 2 si test pas NAN pour avoir l'arrondissement
    - virer les differentes informations dans ville via regex
    
### Step 1: Match uniquement les 1 dans INSEE/INPI

Les siren qui ont uniquement un seul établissement a l'INSEE et à l'INPI.

### Step 2: match Ville/CP/CC

### Step 3: Match adresse

## Description Algorithm

## Description Input

## Description Output

In [ ]:
from inpi_insee import siretisation

%load_ext autoreload
%autoreload 2

In [ ]:
param = {
    'communes_insee': r'data\input\communes_france.csv',
    'upper_word':r'data\input\upper_stop.csv',
    'insee': r'data\input\insee_2017_7480120.gz',
    'inpi_etb': r'data\input\inpi_etb_4979272.gz',
}

In [ ]:
al_siret = siretisation.siretisation_inpi(param)

### Preparation adresse

In [ ]:
test_adress = al_siret.clean_commune()

In [ ]:
test_adress.to_csv('data\input\inpi_etb_cleaned_{}.gz'.format(
    test_adress.shape[0]),
                        compression='gzip', index = False) 

### Match 1-1

In [ ]:
inpi = r'data\input\inpi_etb_cleaned_4979272.gz'
al_siret.match_unique_etb(inpi)

### Merge ville/code postale/ code commune

In [ ]:
%%time
import pandas as pd

inpi_dtype = {
    'siren': 'object',
    'index': 'object',
 'Type': 'object',
 'Adresse_Ligne1': 'object',
 'Adresse_Ligne2': 'object',
 'Adresse_Ligne3': 'object',
 'Code_Postal': 'object',
 'Ville': 'object',
 'Code_Commune': 'object',
 'Pays': 'object',
 'count_initial_inpi': 'int',
 'ncc': 'object',
}

inpi_col =['siren',
        'index',
 'Type',
 'Adresse_Ligne1',
 'Adresse_Ligne2',
 'Adresse_Ligne3',
 'Code_Postal',
 'Ville',
 'Code_Commune',
 'Pays',
 'count_initial_inpi',
 'ncc']

df_ets = r'data\input\unmatched\unmatch_1_unique_siren_1969368.gz'
inpi = al_siret.import_dask(file=df_ets,
                       usecols=inpi_col, dtype=inpi_dtype)

df_nomatch = pd.DataFrame()
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                  option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='1_ville_cp_cc',
        to_csv=True)

### Creation address

In [ ]:
inpi_dtype = {
    'siren': 'object',
    'index': 'object',
 'Type': 'object',
 'Adresse_Ligne1': 'object',
 'Adresse_Ligne2': 'object',
 'Adresse_Ligne3': 'object',
 'Code_Postal': 'object',
 'Ville': 'object',
 'Code_Commune': 'object',
 'Pays': 'object',
 'count_initial_inpi': 'int',
 'ncc': 'object',
}

inpi_col =['siren',
        'index',
 'Type',
 'Adresse_Ligne1',
 'Adresse_Ligne2',
 'Adresse_Ligne3',
 'Code_Postal',
 'Ville',
 'Code_Commune',
 'Pays',
 'count_initial_inpi',
 'ncc']
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_1_ville_cp_cc_744141.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
al_siret.prepare_adress(inpi)

### Loop

In [ ]:
%%time
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


#df_nomatch = pd.DataFrame()
df_match = pd.DataFrame()
for chunk in [0,1
              ,2,3,4,5,6,7,8,9, 10,11,12
             ]:
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(chunk)
    inpi = al_siret.import_dask(file=csv_file,
                       usecols=inpi_col, dtype=inpi_dtype)
    df_input = inpi
    total_match = pd.DataFrame()
    for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
        df_input_ = al_siret.merge_siren_candidat(df_input=df_input,
                                      option=i,
                                      regex_go = True)

        df_input = df_input_['unmatch']
        total_match = total_match.append(df_input_['true_match']) 
    #df_nomatch = df_nomatch.append(df_input)
    df_match = df_match.append(total_match)

In [ ]:
df_temp = pd.DataFrame()
for i in range(0,13):
    csv_file = r'data\input\unmatched\chunk\chunk_{}.gz'.format(i)
    df_temp = df_temp.append(pd.read_csv(csv_file,
                                         dtype= inpi_dtype)
                            )

In [ ]:
299986 / 4979272

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial=df_temp,
        df_inpi_mergeboth=df_match.drop(columns = '_merge'),
        step='3_adress',
        to_csv=True)

### Matching voie

In [ ]:
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

In [ ]:
#temp = al_siret.merge_siren_candidat(df_input=inpi,
#                              regex_go=True, 
#                              matching_voie=True,
#                              option=['ncc', 'libelleCommuneEtablissement'])

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_3_adress_299986.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='4_voie',
        to_csv=True)

### Relax contrainte Regex

Sans numero de voie

In [ ]:
import pandas as pd
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_4_voie_167801.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='5_voie_relax',
        to_csv=True)

Avec la voie

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=True,
        relax_regex = True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_5_relax_116575.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
check = al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='6_voie_relax',
        to_csv=False)

In [ ]:
check.head()

In [ ]:
check['count_initial_inpi'].value_counts()

### Match etablissement principal ouvert

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_6_voie_relax_112847.gz',
                       usecols=inpi_col, dtype=inpi_dtype)

In [ ]:
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i)

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='7_siege_ouvert',
        to_csv=True)

### Complement d adresse

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_7_siege_ouvert_85312.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=True,
        matching_voie=False,
        relax_regex = True,
        siege_etat=True,
        option=i,
        var_adress_insee = 'complementAdresseEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])
    


In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='8_siege_ouvert_complement',
        to_csv=True)

In [ ]:
1- (80474/4979272)

### Test sans regex mais siege_etat

ie si principale-ouvert dans la ville/CP/CC

In [ ]:
import pandas as pd
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}


inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)
df_input = inpi
  
total_match = pd.DataFrame()
for i in [
        ['ncc', 'libelleCommuneEtablissement'],
        ['Code_Postal', 'codePostalEtablissement'],
        ['Code_Commune', 'codeCommuneEtablissement']
    ]:
    df_input_ = al_siret.merge_siren_candidat(
        df_input=df_input,
        regex_go=False,
        matching_voie=False,
        relax_regex = False,
        siege_etat=True,
        option=i,
        var_adress_insee = 'libelleVoieEtablissement')

    df_input = df_input_['unmatch']
    total_match = total_match.append(df_input_['true_match'])

In [ ]:
al_siret.match_unmatch(
        df_inpi_initial= inpi.compute(),
        df_inpi_mergeboth=total_match.drop(columns = '_merge'),
        step='9_siege_ouvert_no_regex',
        to_csv=True)

In [ ]:
round(1 - (68131 / 4979272),3)

## Amelioration matching

In [ ]:
import pandas as pd
insee_dtype = {
    'siren': 'object',
    'siret': 'object',
    'dateCreationEtablissement': 'object',
    "etablissementSiege": "object",
    "etatAdministratifEtablissement": "object",
    'complementAdresseEtablissement': 'object',
    'numeroVoieEtablissement': 'object',
    'indiceRepetitionEtablissement': 'object',
    'typeVoieEtablissement': 'object',
    'libelleVoieEtablissement': 'object',
    'codePostalEtablissement': 'object',
    'libelleCommuneEtablissement': 'object',
    'libelleCommuneEtrangerEtablissement': 'object',
    'distributionSpecialeEtablissement': 'object',
    'codeCommuneEtablissement': 'object',
    'codeCedexEtablissement': 'object',
    'libelleCedexEtablissement': 'object',
    'codePaysEtrangerEtablissement': 'object',
    'libellePaysEtrangerEtablissement': 'object',
    'count_initial_insee': 'int'
}

insee_col = ['siren',
             'siret',
             'dateCreationEtablissement',
             "etablissementSiege",
             "etatAdministratifEtablissement",
             'complementAdresseEtablissement',
             'numeroVoieEtablissement',
             'indiceRepetitionEtablissement',
             'typeVoieEtablissement',
             'libelleVoieEtablissement',
             'codePostalEtablissement',
             'libelleCommuneEtablissement',
             'libelleCommuneEtrangerEtablissement',
             'distributionSpecialeEtablissement',
             'codeCommuneEtablissement',
             'codeCedexEtablissement',
             'libelleCedexEtablissement',
             'codePaysEtrangerEtablissement',
             'libellePaysEtrangerEtablissement',
             'count_initial_insee']
inpi_col = ['siren',
            'index',
            'Type',
            'Adresse_Ligne1',
            'Adresse_Ligne2',
            'Adresse_Ligne3',
            'Code_Postal',
            'Ville',
            'Code_Commune',
            'Pays',
            'count_initial_inpi',
            'ncc',
            'Adress_new',
            'Adresse_new_clean_reg']

inpi_dtype = {
    'siren': 'object',
    'index': 'int',
    'Type': 'object',
    'Adresse_Ligne1': 'object',
    'Adresse_Ligne2': 'object',
    'Adresse_Ligne3': 'object',
    'Code_Postal': 'object',
    'Ville': 'object',
    'Code_Commune': 'object',
    'Pays': 'object',
    'count_initial_inpi': 'int',
    'ncc': 'object',
    'Adress_new': 'object',
    'Adresse_new_clean_reg': 'object'
}

insee = al_siret.import_dask(file=r'data\input\insee_2017_7480120.gz',
                             usecols=insee_col, dtype=insee_dtype)

#insee['etatAdministratifEtablissement'].value_counts().compute()
inpi = al_siret.import_dask(file=r'data\input\unmatched\unmatch_8_siege_ouvert_complement_80474.gz',
                       usecols=inpi_col, dtype=inpi_dtype)


In [ ]:
inpi.head()

In [ ]:
insee[
    (insee['siren'].isin(['515226462']))
#& (insee['libelleVoieEtablissement'].isin(['JACQUARD']))
].compute()

In [ ]:
inpi[inpi['siren'].isin(['515226462'])].compute()

In [ ]:
inpi.loc[inpi['count_initial_inpi'].isin([1])].head(10)

In [ ]:
inpi.loc[inpi['siren'].isin(['322385949'])].compute()

In [ ]:
inpi['Type'].value_counts().compute()

In [ ]:
insee[insee['siren'].isin(['322385949'])].compute()

Verification conservative:

- A l'INSEE, si etatAdministratifEtablissement == A & etablissementSiege == True, alors on fait le sous ensemle
- Verifier avec les dates de création de l'établissement

In [ ]:
insee.loc[
            (insee['etablissementSiege'].isin(['true'])) 
            & (insee['etatAdministratifEtablissement'].isin(['A']))
            ].compute() 

In [ ]:
1 - (112847 / 4979272)

In [ ]:
amelioration possible:
    
- 381980788 -> Zone Artisanale du Creusât ->transformer en ZA